In [1]:
from Pseudo_Goldstone.Get_Feynman_Rules import Make_Feynman_Rules, VertexValue
from Pseudo_Goldstone import Tuples
import os
import sympy as sp
import numpy as np
import re
from IPython.display import Math



def ApplyTex(expr):
    tmpE=sp.latex(expr)
    for i in SubsTex:
        tmpE=sp.latex(tmpE).replace(i[0],i[1])
        
    return tmpE    

def ApplyTexfields(expr):
    tmpE=sp.latex(map(eval,expr))
    for i in SubsTex:
        tmpE=sp.latex(tmpE).replace(i[0],i[1])
        
    return tmpE    



def MathNice(expr):

    return Math(ApplyTex(expr)) 

In [6]:
dimN=2
gauge='feyn'
Dir=r'Frules//SU{}/'.format(dimN)
I=sp.I
sqrt=lambda x: sp.sqrt(x)
conjugate=lambda x: sp.conjugate(x)
Abs=lambda x: sp.Abs(x)

Read_Dict={}
for i in os.listdir(Dir):
    if gauge in i:
        
        a = re.search(r'.*\.(.*)',i).group(1)
        Frules_in=open(Dir+'/'+str(i),'r')
        if a=='vrt':
            point= re.search(r'.*\_(.)-point.',i).group(1)
            Read_Dict[point+'-point']=Frules_in.read().split('\n')
            Read_Dict[point+'-point'].remove('')
            Read_Dict[point+'-point']=np.array(Read_Dict[point+'-point'])
        else:
            Read_Dict[a]=Frules_in.read().split('\n')
            for i in range(100):
                try:
                    Read_Dict[a].remove('')
                except:
                    break
            Read_Dict[a]=np.array(Read_Dict[a])
        Frules_in.close()


for i in Read_Dict['fields']:
    sp.var(i)
#Read_Dict.pop('fields',None)#I need the fields

param_it=iter(Read_Dict['parameters'])
while True:
    try:
        Sym=param_it.next()
        tmp=param_it.next()
        
        assume=re.search('\[(.*)\,(.*)\,(.*)\]',tmp)
        sp.var(Sym, complex=assume.group(1),real=assume.group(2),positive=assume.group(3) )
        #print eval(Sym), eval(Sym+'.assumptions0') 
        #print '..........................'
        
    except StopIteration:
        Read_Dict.pop('parameters',None)
        break

SubsTex=Read_Dict['subs'][:]
SubsTex=np.array(list(reversed(SubsTex)))

Read_Dict.pop('subs',None)
def Convert(line):
    tmp=line.replace(' ','').replace('\n','').split('|')
    tmp[0]=tmp[0].replace('(','_._').replace(')','_._').replace(',','_._').split('_._')
    while True:
        try:
            tmp[0].remove('')
        except ValueError:
            break
    p=len(tmp[0])
    if p==2:
        #-1 because I need to convert it in order to get the correct result (from VertexValue) after Make_Feynman_Rules
        return [ map(eval,tmp[0]), eval(tmp[1]),-1 ]
    else:
        #I because I need to convert it in order to get the correct result (from VertexValue) after Make_Feynman_Rules
        return [ map(eval,tmp[0]), eval(tmp[1]),I ]

VConvert=np.vectorize(Convert)

Frules={}
Read_Dict_Tmp=Read_Dict.copy()
for i in Read_Dict:
    if '-point' in i:
        j=eval(i.replace('-point',''))
        Frules[j]=VConvert(np.array(Read_Dict[i]))
        Read_Dict_Tmp.pop(i,None)
        
Read_Dict=Read_Dict_Tmp      
Make_Feynman_Rules(Frules)


tmp=np.array([])
for i in SubsTex:
    i=i.replace(' ','').split('|')
    tmp=np.append(i,tmp)
SubsTex=tmp.reshape(len(tmp)/2,2)[:]
del tmp

In [7]:
Mass=[]
for k in [2]:
    for i in Tuples(Read_Dict['fields'],k):
        s='VertexValue('
        it=iter(i)

        while True:
            try:
                j=it.next()
                s+=j+','
            except:

                s+=')'
                break
        if   eval(s) !=0:   
            Mass+= ( i, sp.simplify(eval(s)) )
        
Mass=np.array(Mass).reshape(len(Mass)/2,2) 

Frules=[]
for k in [3,4]:
    for i in Tuples(Read_Dict['fields'],k):
        s='VertexValue('
        it=iter(i)

        while True:
            try:
                j=it.next()
                s+=j+','
            except:

                s+=')'
                break
        if   eval(s) !=0:   
            Frules+= ( i, sp.simplify(eval(s)) )
        

Frules=np.array(Frules).reshape(len(Frules)/2,2) 

In [8]:
a=open('HTML/subs.dat','r')
tmp=a.read().split('\n')
a.close()
Relations=[]
tmp.remove('')
for i in tmp:
    a=i.replace(' ','').split('|')
    sp.var(a[0])
    Relations+=[(a[0],a[1])] 
    
sp.var('MH MS0');

In [9]:
in_html=open('HTML/misc','r')
file_html=open('HTML/Frules/Frules.html','w')
file_html.write( in_html.read() )
in_html.close()

file_html.write('<body> \n')


file_html.write('<p>')
#FRules
file_html.write('\\begin{array}{|l|l|}')
file_html.write(  ' {\\rm \\bf Vertex}'+ ' \qquad \qquad &' + '{\\rm \\bf Feynman \; Rule}'+'  \\\\ \hline \\\\' ) 
for i in Frules:
    
    file_html.write(  ApplyTexfields(i[0])+ ' \qquad \qquad &' + ApplyTex(i[1])+' \\\\ \hline \\\\' ) 
file_html.write('\\end{array}')
file_html.write('<p>')
#Mass Matrix entries
file_html.write('  \\begin{array}{ll)}')
file_html.write(  ' {\\rm \\bf Particles}'+ '  &' + '{\\rm \\bf Mass \; Matrix \; element}'+'  \\\\ \hline \\\\' ) 
for i in Mass:
    file_html.write(  ApplyTexfields(i[0])+ ' \qquad \qquad &' + ApplyTex(i[1])+' \\\\ \hline \\\\' ) 
    
file_html.write('\\end{array} ')
#relations
file_html.write(  '${\\rm \\bf Relations: }$' ) 
file_html.write('  \\begin{align}')

for i in Relations:
    file_html.write(  '&' + ApplyTex(eval(i[0]))+ ' = ' + ApplyTex(eval(i[1]))+' \\\\ ' ) 
    
file_html.write('\\end{align} ')



file_html.write('\n </body> \n </html>')
file_html.close()

In [10]:
in_html=open('HTML/misc','r')
file_html=open('HTML/L.html','w')
file_html.write( in_html.read() )
in_html.close()

file_html.write('<body> \n')


file_html.write('<p>')

file_html.write('  \\begin{align}')
file_html.write(ApplyTex(eval(Read_Dict['LMassInt'][0])))
file_html.write('  \\end{align}')



file_html.write('\n </body> \n </html>')
file_html.close()